<img src="https://files.training.databricks.com/images/DeltaLake-logo.png" width="80px"/>

# Unifying Structured Streaming with Batch Jobs with Delta Lake

In this notebook, we will explore combining streaming and batch processing with a single pipeline. We will begin by defining the following logic:

- ingest streaming JSON data from disk and write it to a Delta Lake Table `/activity/Bronze`
- perform a Stream-Static Join on the streamed data to add additional geographic data
- transform and load the data, saving it out to our Delta Lake Table `/activity/Silver`
- summarize the data through aggregation into the Delta Lake Table `/activity/Gold/groupedCounts`
- materialize views of our gold table through streaming plots and static queries

We will then demonstrate that by writing batches of data back to our bronze table, we can trigger the same logic on newly loaded data and propagate our changes automatically.

In [2]:
%run "./Includes/Classroom-Setup"

-sandbox
## Set up relevant Delta Lake paths

These paths will serve as the file locations for our Delta Lake tables.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Each streaming write has its own checkpoint directory.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> You cannot write out new Delta files within a repository that contains Delta files. Note that our hierarchy here isolates each Delta table into its own directory.

In [4]:
activityPath = userhome + "/activity"

activityBronzePath = activityPath + "/Bronze"
activityBronzeCheckpoint = activityBronzePath + "/checkpoint"

activitySilverPath = activityPath + "/Silver"
activitySilverCheckpoint = activitySilverPath + "/checkpoint"

activityGoldPath = activityPath + "/Gold"
groupedCountPath = activityGoldPath + "/groupedCount"
groupedCountCheckpoint = groupedCountPath + "/checkpoint"

## Reset Pipeline

To reset the pipeline, run the following:

In [6]:
dbutils.fs.rm(activityPath, True)

Out[6]: False

-sandbox
## Datasets Used
This notebook will consume cell phone accelerometer data. Records have been downsampled so that the streaming data represents less than 3% of the total data being produced. The remainder will be processed as batches.

The following fields are present:

- `Index`
- `Arrival_Time`
- `Creation_Time`
- `x`
- `y`
- `z`
- `User`
- `Model`
- `Device`
- `gt`
- `geolocation`

## Define Schema

For streaming jobs, we need to define our schema before we start.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We'll reuse this same schema later in the notebook to define our batch processing, which will eliminate the jobs triggered by eliminating a file scan AND enforce the schema that we've defined here.

In [8]:
from pyspark.sql.types import StructField, StructType, LongType, StringType, DoubleType

schema = StructType([
  StructField("Arrival_Time",LongType()),
  StructField("Creation_Time",LongType()),
  StructField("Device",StringType()),
  StructField("Index",LongType()),
  StructField("Model",StringType()),
  StructField("User",StringType()),
  StructField("geolocation",StructType([
    StructField("city",StringType()),
    StructField("country",StringType())
  ])),
  StructField("gt",StringType()),
  StructField("id",LongType()),
  StructField("x",DoubleType()),
  StructField("y",DoubleType()),
  StructField("z",DoubleType())
])

-sandbox

### Define Streaming Load from Files in Blob

Our streaming source directory has 36 JSON files of 5k records each saved in a repository. Here, we'll trigger processing on files one at a time. 

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> In a production setting, this same logic would allow us to only read new files written to our source directory. We could define `maxFilesPerTrigger` to control the amount of data we consume with each load, or omit this option to consume all new data on disk since the last time the stream has processed.

In [10]:
rawEventsDF = (spark
  .readStream
  .format("json")
  .schema(schema)
  .option("maxFilesPerTrigger", 1)
  .load("/mnt/training/definitive-guide/data/activity-json/streaming"))

-sandbox
### WRITE Stream using Delta Lake

#### General Notation
Use this format to write a streaming job to a Delta Lake table.

<pre>
(myDF
  .writeStream
  .format("delta")
  .option("checkpointLocation", checkpointPath)
  .outputMode("append")
  .start(path)
)
</pre>

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> While we _can_ write directly to tables using the `.table()` notation, this will create fully managed tables by writing output to a default location on DBFS. This is not best practice for production jobs.

#### Output Modes
Notice, besides the "obvious" parameters, specify `outputMode`, which can take on these values
* `append`: add only new records to output sink
* `complete`: rewrite full output - applicable to aggregations operations

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> At present, `update` mode is **not** supported for streaming Delta jobs.

#### Checkpointing

When defining a Delta Lake streaming query, one of the options that you need to specify is the location of a checkpoint directory.

`.writeStream.format("delta").option("checkpointLocation", <path-to-checkpoint-directory>) ...`

This is actually a structured streaming feature. It stores the current state of your streaming job.

Should your streaming job stop for some reason and you restart it, it will continue from where it left off.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> If you do not have a checkpoint directory, when the streaming job stops, you lose all state around your streaming job and upon restart, you start from scratch.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Also note that every streaming job should have its own checkpoint directory: no sharing.

In [12]:
(rawEventsDF
  .writeStream
  .format("delta")
  .option("checkpointLocation", activityBronzeCheckpoint)
  .outputMode("append")
  .start(activityBronzePath))

Out[9]: <pyspark.sql.streaming.StreamingQuery at 0x7f6df804c450>

### Load Static Lookup Table

Before enriching our bronze data, we will load a static lookup table for our country codes.

Here, we'll use a parquet file that contains countries and their associated codes and abbreviations.

While we can load this as a table (which will copy all files to the workspace and make it available to all users), here we'll manipulate it as a DataFrame.

In [14]:
from pyspark.sql.functions import col

geoForLookupDF = (spark
  .read
  .format("parquet")
  .load("/mnt/training/countries/ISOCountryCodes/ISOCountryLookup.parquet/")
  .select(col("EnglishShortName").alias("country"), col("alpha3Code").alias("countryCode3")))

-sandbox

## Create QUERY tables (aka "silver tables")

Our current bronze table contains nested fields, as well as time data that has been encoded in non-standard unix time (`Arrival_Time` is encoded as milliseconds from epoch, while `Creation_Time` records nanoseconds between record creation and receipt). 

We also wish to enrich our data with 3 letter country codes for mapping purposes, which we'll obtain from a join with our `geoForLookupDF`.

In order to parse the data in human-readable form, we create query/silver tables out of the raw data.

We will stream from our previous file write, define transformations, and rewrite our data to disk.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Notice how we do not need to specify a schema when loading Delta files: it is inferred from the metadata!

The fields of a complex object can be referenced with a "dot" notation as in:

`col("geolocation.country")`


A large number of these fields/columns can become unwieldy.

For that reason, it is common to extract the sub-fields and represent them as first-level columns as seen below:

NOTE: You will not be able to run this command until the `rawEventsDF` has initialized.

In [17]:
from pyspark.sql.functions import from_unixtime

parsedEventsDF = (spark.readStream
  .format("delta")
  .load(activityBronzePath)
  .select(from_unixtime(col("Arrival_Time")/1000).alias("Arrival_Time").cast("timestamp"),
          (col("Creation_Time")/1E9).alias("Creation_Time").cast("timestamp"),
          col("Device"),
          col("Index"),
          col("Model"),
          col("User"),
          col("gt"),
          col("x"),
          col("y"),
          col("z"),
          col("geolocation.country").alias("country"),
          col("geolocation.city").alias("city"))
  .join(geoForLookupDF, ["country"], "left"))

### Write to QUERY Tables (aka "silver tables")

In [19]:
(parsedEventsDF
  .writeStream
  .format("delta")
  .option("checkpointLocation", activitySilverCheckpoint)
  .outputMode("append")
  .start(activitySilverPath))

Out[12]: <pyspark.sql.streaming.StreamingQuery at 0x7f6df839e310>

See contents of Silver directory.

In [21]:
dbutils.fs.ls(activitySilverPath)

Out[13]: [FileInfo(path='dbfs:/user/lregoazd@outlook.com/activity/Silver/_delta_log/', name='_delta_log/', size=0),
 FileInfo(path='dbfs:/user/lregoazd@outlook.com/activity/Silver/checkpoint/', name='checkpoint/', size=0),
 FileInfo(path='dbfs:/user/lregoazd@outlook.com/activity/Silver/part-00000-a55fb8d0-c22e-4637-8c96-d4198eb555e3-c000.snappy.parquet', name='part-00000-a55fb8d0-c22e-4637-8c96-d4198eb555e3-c000.snappy.parquet', size=1288095),
 FileInfo(path='dbfs:/user/lregoazd@outlook.com/activity/Silver/part-00001-7ff60690-4795-4a62-9176-eb5766958a1b-c000.snappy.parquet', name='part-00001-7ff60690-4795-4a62-9176-eb5766958a1b-c000.snappy.parquet', size=1170933),
 FileInfo(path='dbfs:/user/lregoazd@outlook.com/activity/Silver/part-00002-a83f81cc-3f65-42a6-99b0-b45495c71bcb-c000.snappy.parquet', name='part-00002-a83f81cc-3f65-42a6-99b0-b45495c71bcb-c000.snappy.parquet', size=1110853),
 FileInfo(path='dbfs:/user/lregoazd@outlook.com/activity/Silver/part-00003-b9006580-0dbb-4c34-8d06-657da66d8ab3-c000.snappy.parquet', name='part-00003-b9006580-0dbb-4c34-8d06-657da66d8ab3-c000.snappy.parquet', size=949320)]

-sandbox
#### See list of active streams.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You should currently see two active streams, one for each streaming write that you've triggered. If you have called `display` on either of your streaming DataFrames, you will see an additional stream, as `display` writes the stream to memory.

In [23]:
for s in spark.streams.active:
  print(s.id)

b05df5b8-db01-48fd-bcd6-bf00d326211d
63c93ed9-f2ee-4ad6-a7d2-c54c61f17a3c

-sandbox
### Gold Table: Grouped Count of Events

Here we read a stream of data from `activitySilverPath` and write another stream to `activityGoldPath/groupedCount`.

The data consists of a total counts of all event, grouped by `hour`, `gt`, and `countryCode3`.

Performing this aggregation allows us to reduce the total number of rows in our table from hundreds of thousands (or millions, once we've loaded our batch data) to dozens.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Notice that we're writing to a named directory within our gold path. If we wish to define additional aggregations, we would organize these parallel to thie directory to avoid metadata write conflicts.

In [25]:
from pyspark.sql.functions import window, hour

(spark.readStream
  .format("delta")
  .load(activitySilverPath)
  .groupBy(window("Arrival_Time", "60 minute"),"gt", "countryCode3")
  .count()
  .withColumn("hour",hour(col("window.start")))
  .drop("window")
  .writeStream
  .format("delta")
  .option("checkpointLocation", groupedCountCheckpoint)
  .outputMode("complete")
  .start(groupedCountPath))

Out[15]: <pyspark.sql.streaming.StreamingQuery at 0x7f6df80657d0>

### CREATE A Table Using Delta Lake

Create a table called `gt_count` using `DELTA` out of the above data.

NOTE: You will not be able to run this command until the `activityCountsQuery` has initialized.

In [27]:
spark.sql("""
  DROP TABLE IF EXISTS grouped_count
""")
spark.sql("""
  CREATE TABLE grouped_count
  USING DELTA
  LOCATION '{}'
""".format(groupedCountPath))

Out[16]: DataFrame[]

-sandbox
#### Important Considerations for `complete` Output with Delta

When using `complete` output mode, we rewrite the entire state of our table each time our logic runs. While this is ideal for calculating aggregates, we **cannot** read a stream from this directory, as Structured Streaming assumes data is only being appended in the upstream logic.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Certain options can be set to change this behavior, but have other limitations attached. For more details, refer to [Delta Streaming: Ignoring Updates and Deletes](https://docs.databricks.com/delta/delta-streaming.html#ignoring-updates-and-deletes).

The gold Delta table we have just registered will perform a static read of the current state of the data each time we run the following query.

In [29]:
%sql
SELECT * FROM grouped_count

gt,countryCode3,count,hour
stairsdown,IND,440,12
stairsdown,DEU,2542,13
stairsdown,NGA,2810,11
stairsdown,USA,2468,12
stairsdown,AUS,2461,13
stairsdown,IND,1752,13
stairsdown,FRA,2650,14
stairsdown,BRA,2521,14
bike,NGA,3204,11
sit,NGA,2970,11


-sandbox
### Materialized View: Windowed Count of Hourly `gt` Events

Plot the occurrence of all events grouped by `gt`.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Because we're using `complete` output mode for our gold table write, we cannot define a streaming plot on these files.

Instead, we'll define a temp table based on the files written to our silver table. We will them use this table to execute our streaming queries.

In order to create a LIVE bar chart of the data, you'll need to fill out the <b>Plot Options</b> as shown:

<div><img src="https://files.training.databricks.com/images/eLearning/Delta/ch5-plot-options.png"/></div><br/>

### Note on Gold Tables & Materialized Views

When we call `display` on a streaming DataFrame or execute a SQL query on a streaming view, we are using memory as our sink. 

In this case, we have already calculated all the values necessary to materialize our streaming view above in the gold table we've written to disk. 

**However**, we re-execute this logic on our silver table to generate streaming views, as structured streaming will not support reads from upstream files that have beem overwritten.

In [31]:
(spark.readStream
  .format("delta")
  .load(activitySilverPath)
  .createOrReplaceTempView("query_table")
)

In [32]:
%sql
SELECT gt, HOUR(Arrival_Time) hour, COUNT(*) total_events
FROM query_table
GROUP BY gt, HOUR(Arrival_Time)
ORDER BY hour

gt,hour,total_events
null,10,140087
bike,10,88920
sit,10,115241
walk,10,126856
stand,10,105632
stairsdown,10,79414
stairsup,10,77237
bike,11,105874
null,11,100736
sit,11,117042


## Batch Load Data into Bronze Table

We can use the same pipeline to process batch data.

By loading our raw data into our bronze table, we will push it through our already running streaming logic.

Here, we'll run 4 batches of around 170k records. We can track each batch through our streaming plots above.

In [34]:
for batch in range(4):
  (spark
    .read
    .format("json")
    .schema(schema)
    .load("/mnt/training/definitive-guide/data/activity-json/batch-{}".format(batch))
    .write
    .format("delta")
    .mode("append")
    .save(activityBronzePath))

Note that even on our small cluster, we can pass a batch of over 5 million records through our logic above without problems.

In [36]:
(spark
  .read
  .format("json")
  .schema(schema)
  .load("/mnt/training/definitive-guide/data/activity-json/batch")
  .write
  .format("delta")
  .mode("append")
  .save(activityBronzePath))

-sandbox

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> While our streaming materialized view above updates as data flows in, we can also easily generate this view from our `grouped_count` table. 

We will need to re-run this query each time we wish to update the data. Run the below query now, and then after your batch has finished processing.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> The state reflected in a query on a registered Delta table will always reflect the most recent valid state of the files.

In [38]:
%sql
SELECT * FROM grouped_count

gt,countryCode3,count,hour
stairsdown,IND,440,12
stairsdown,DEU,2542,13
stairsdown,NGA,2810,11
stairsdown,USA,2468,12
stairsdown,AUS,2461,13
stairsdown,IND,1752,13
stairsdown,FRA,2650,14
stairsdown,BRA,2521,14
sit,NGA,2970,11
bike,NGA,3204,11


## Wrapping Up

Finally, make sure all streams are stopped.

In [40]:
for s in spark.streams.active:
    s.stop()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1947196994887737> in <module> 
 1 for s in spark . streams . active : 
 ----> 2 s . stop ( ) 

 /databricks/spark/python/pyspark/sql/streaming.py in stop (self) 
 151 """Stop this streaming query.
 152 """
 --> 153 self . _jsq . stop ( ) 
 154 
 155 @ since ( 2.1 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 126 def deco ( * a , ** kw ) : 
 127 try : 
 --> 128 return f ( * a , ** kw ) 
 129 except py4j . protocol . Py4JJavaError as e : 
 130 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o979.stop.
: java.util.concurrent.TimeoutException: Stream Execution thread for stream display_query_1 [id = 47de691f-c125-469a-bcfd-e03445c95bcc, runId = 6377542a-6c27-4976-93a9-3fc3175c53ad] failed to stop within 15000 milliseconds (specified by spark.sql.streaming.stopTimeout). See the cause on what was being executed in the streaming query thread.
	at org.apache.spark.sql.execution.streaming.StreamExecution.interruptAndAwaitExecutionThreadTermination(StreamExecution.scala:488)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution.stop(MicroBatchExecution.scala:172)
	at org.apache.spark.sql.execution.streaming.StreamingQueryWrapper.stop(StreamingQueryWrapper.scala:67)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: The stream thread was last executing:
	at java.net.SocketInputStream.socketRead0(Native Method)
	at java.net.SocketInputStream.socketRead(SocketInputStream.java:116)
	at java.net.SocketInputStream.read(SocketInputStream.java:171)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at sun.security.ssl.InputRecord.readFully(InputRecord.java:465)
	at sun.security.ssl.InputRecord.read(InputRecord.java:503)
	at sun.security.ssl.SSLSocketImpl.readRecord(SSLSocketImpl.java:990)
	at sun.security.ssl.SSLSocketImpl.readDataRecord(SSLSocketImpl.java:948)
	at sun.security.ssl.AppInputStream.read(AppInputStream.java:105)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read1(BufferedInputStream.java:286)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:345)
	at sun.net.www.http.HttpClient.parseHTTPHeader(HttpClient.java:735)
	at sun.net.www.http.HttpClient.parseHTTP(HttpClient.java:678)
	at sun.net.www.protocol.http.HttpURLConnection.getInputStream0(HttpURLConnection.java:1593)
	at sun.net.www.protocol.http.HttpURLConnection.access$200(HttpURLConnection.java:92)
	at sun.net.www.protocol.http.HttpURLConnection$9.run(HttpURLConnection.java:1490)
	at sun.net.www.protocol.

## Summary

Delta Lake is ideally suited for use in streaming data lake contexts.

Use the Delta Lake architecture to craft raw, query, and summary tables to produce beautiful visualizations of key business metrics.

## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/delta-streaming.html#as-a-sink" target="_blank">Delta Streaming Write Notation</a>
* <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#" target="_blank">Structured Streaming Programming Guide</a>
* <a href="https://www.youtube.com/watch?v=rl8dIzTpxrI" target="_blank">A Deep Dive into Structured Streaming</a> by Tagatha Das. This is an excellent video describing how Structured Streaming works.
* <a href="http://lambda-architecture.net/#" target="_blank">Lambda Architecture</a>
* <a href="https://bennyaustin.wordpress.com/2010/05/02/kimball-and-inmon-dw-models/#" target="_blank">Data Warehouse Models</a>
* <a href="https://people.apache.org//~pwendell/spark-nightly/spark-branch-2.1-docs/latest/structured-streaming-kafka-integration.html#" target="_blank">Reading structured streams from Kafka</a>
* <a href="http://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html#creating-a-kafka-source-stream#" target="_blank">Create a Kafka Source Stream</a>
* <a href="https://docs.databricks.com/delta/delta-intro.html#case-study-multi-hop-pipelines#" target="_blank">Multi Hop Pipelines</a>